Installing Required Libraries


In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade torch

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to c:\users\user\appdata\local\temp\pip-req-build-ijhuwp0c
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\USER\AppData\Local\Temp\pip-req-build-ijhuwp0c'


In [7]:
pip install git+https://github.com/librosa/librosa

  Cloning https://github.com/librosa/librosa to c:\users\user\appdata\local\temp\pip-req-build-5s_044du
  Resolved https://github.com/librosa/librosa to commit 9ec4f4b73208159454477df7fff8ff60cdf3b47a
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for librosa: filename=librosa-0.11.0rc1-py3-none-any.whl size=262938 sha256=f151b2cde86a10a3214e93296c9a7299a818f254e63c28fe2c5f003b02288fd6
  Stored in directory: C:\Users\USER\AppData\Local\Temp\pip-ephem-wheel-cache-avusuxrk\wheels\d6\c4\f0\6bffc777ecfbd8e6852bf0bc2d87ac3972ab9694b080fe30b5
Successfully built librosa
  Attempting uninstall: librosa
    Found existing installation: librosa 0.11.0rc1
    Uninstalling librosa-0.11.0rc1:
      Succes

  Running command git clone --filter=blob:none --quiet https://github.com/librosa/librosa 'C:\Users\USER\AppData\Local\Temp\pip-req-build-5s_044du'
  Running command git submodule update --init --recursive -q


Importing Required Libraries


In [11]:
import torch
import whisper
import librosa
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd
import jiwer  
import AudioInfo
import yt_dlp
import os
import json
import soundfile as sf
import noisereduce as nr
import tkinter
from tkinter import ttk, scrolledtext
from urllib.parse import urlparse
from youtube_transcript_api import YouTubeTranscriptApi

d:\FE\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading the models


In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_t = whisper.load_model("small").to(device)

In [1]:
def downloadAudioFromYoutubeLink(_targetUrl, _outputFile="audio.mp3"):
    ydl_opts = {
        'format': 'bestaudio/best',  # Choose the best available audio format
        'outtmpl': _outputFile,  # Set the output filename
        'noplaylist': True, # only download single video if it's a playlist
    }
    try:
        if os.path.exists(_outputFile):
            os.remove(_outputFile)
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([_targetUrl])
        print(f"Audio downloaded successfully to {_outputFile}")
        return _outputFile 
    except Exception as e:
        print(f"An error occurred: {e}")
        return None 

In [6]:
def popupInputLinkFileName():
    def getInfo():
        global video_url, file_path
        video_url = entryLink.get()
        file_path = entryFileName.get()
        if file_path is None or video_url is None:  # Check if file_path is None
            ket_qua_label.config(text="Tên file và link không được để trống.")
            return
        if not file_path.endswith('.mp3'):
            ket_qua_label.config(text="Tên file phải kết thúc bằng .mp3")
            return
        video_url = video_url[0:video_url.index('&')] if '&' in video_url else video_url
        file_path = downloadAudioFromYoutubeLink(video_url, file_path)
        if file_path == None:
            ket_qua_label.config(text="Link không đúng.")
            return
        ket_qua_label.config(text=f"Link: {video_url}\n Tên file: {file_path}")
        window.destroy()

    window = tkinter.Tk()
    window.title("Nhập thông tin")

    # Nhãn và hộp nhập liệu cho tên
    tkinter.Label(window, text="Đường dẫn liên kết:").pack()
    entryLink = tkinter.Entry(window)
    entryLink.pack()

    # Nhãn và hộp nhập liệu cho tuổi
    tkinter.Label(window, text="Tên file mong muốn:").pack()
    entryFileName = tkinter.Entry(window)
    entryFileName.pack()
    entryFileName.setvar("abc")

    # Nút bấm để lấy thông tin
    button = tkinter.Button(window, text="Lấy thông tin", command=getInfo)
    button.pack()

    # Nhãn để hiển thị kết quả
    ket_qua_label = tkinter.Label(window, text='')
    ket_qua_label.pack()

    window.mainloop()
    return video_url, file_path

In [9]:
def boLocNhieu(_filePath):
    audio, sr = librosa.load(_filePath, sr=16000)
    
    noise_part = audio[-16000:]
    
    denoised_audio = nr.reduce_noise(y=audio, sr=sr, y_noise=noise_part, prop_decrease=0.5)

    sf.write(_filePath.replace('.mp3', '_1.mp3'), denoised_audio, sr)
     # Vẽ đồ thị
    plt.figure(figsize=(12, 6))

    plt.subplot(2, 1, 1)
    librosa.display.waveshow(audio, sr=sr, alpha=0.5, color='r')
    plt.title("Tín hiệu gốc (Original Audio)")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")

    plt.subplot(2, 1, 2)
    librosa.display.waveshow(denoised_audio, sr=sr, alpha=0.5, color='b')
    plt.title("Tín hiệu sau lọc nhiễu (Denoised Audio)")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")

    plt.tight_layout()
    plt.show()

In [12]:
try:
    audio_data = whisper.load_audio(file_path)
    result = model_t.transcribe(file_path, fp16 = True if device == 'cuda' else False )
    print("Audio loaded successfully!")
except FileNotFoundError as e:
    print(f"Error: {e}")
    print("Please make sure FFmpeg is installed and added to your PATH.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

An unexpected error occurred: name 'file_path' is not defined
